In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
from pathlib import Path
from PIL import Image
import json

In [8]:
root_path = Path('./data/')

In [10]:
style_df = pd.read_csv(root_path / 'metadata' / 'style.csv')

content_df = pd.read_csv(root_path / 'metadata' / 'content.csv')

In [13]:
content_df.head()

,Title,Author,Orientation,Resolution,File name,Notes
0,Mona Lisa,Leonardo Da Vinci,Portrait,1024x1526,mona_lisa.jpeg,NaN
1,The Last Supper,Leonardo Da Vinci,Landscape,1200x600,last_supper.jpeg,NaN
2,Persistence Of Memory,Salvador Dalì,Landscape,1668 × 1200,persistence_of_the_memory.jpg,NaN
3,The Birth Of Venus,Sandro Botticelli,Landscape,1200 x 754,the_birth_of_venus.jpg,NaN
4,The Dance,Henri Matisse,Landscape,2348 × 1587,the_dance.jpg,NaN


In [14]:
style_df.head()

,Title,Author,Resolution,File name,Notes
0,Flowers,Ben Giles,512x512,flowers.jpg,NaN
1,Edtaonisl,Francis Picabia,800x776,edtaonisl.jpg,NaN
2,Udnie,Francis Picabia,512x512,udnie.jpg,NaN
3,Tahiti,Paul Gauguin,1220x687,tahiti.jpg,NaN
4,Red Canna,Georgia O'Keeffe,600x800,red_canna.png,NaN


In [15]:
content_path = root_path / 'content-images'
content_landscape_path = content_path / 'landscapes'
content_portrait_path = content_path / 'portraits'

style_path = root_path / 'style-images'

In [ ]:
for i, content in content_df.iterrows():
    for j, style in style_df.iterrows():
        
        if content['Orientation'] == 'Portrait':
            content_img_path = content_portrait_path / content['File name']
        else:
            content_img_path = content_landscape_path / content['File name']
            
        style_img_path = style_path / style['File name']
                
        metadata = {
            "description": "TBD",
            "image": "TBD",
            "name" : f"{content['Title']} + {style['Title']}",
            "animation_url": "TBD",
            "attributes": [
                {
                    "trait_type": "Content",
                    "value": content['Title']
                },
                {
                    "trait_type": "Content Author",
                    "value": content['Author']
                },
                {
                    "trait_type": "Style",
                    "value": style['Title']
                },
                {
                    "trait_type": "Style Author",
                    "value": style['Author']
                },
                {
                    "trait_type": "Orientation",
                    "value": content['Orientation']
                }
                
            ]
        }
        
        print(f"Processing content image: {content['File name']}")
        print(f"Processing style: {style['File name']}")
        print(json.dumps(metadata, indent=4))
        
        #content_img = Image.open(content_img_path)
        #style_img = Image.open(style_img_path)
        #content_img.show()
        #style_img.show()
        
        !python3 neural_style_transfer.py --content_img_name {content['File name']} --style_img_name {style['File name']}
        
        print()
    
    break
    print()

Processing content image: mona_lisa.jpeg
Processing style: flowers.jpg
{
    "description": "TBD",
    "image": "TBD",
    "name": "Mona Lisa + Flowers",
    "animation_url": "TBD",
    "attributes": [
        {
            "trait_type": "Content",
            "value": "Mona Lisa"
        },
        {
            "trait_type": "Content Author",
            "value": "Leonardo Da Vinci"
        },
        {
            "trait_type": "Style",
            "value": "Flowers"
        },
        {
            "trait_type": "Style Author",
            "value": "Ben Giles"
        },
        {
            "trait_type": "Orientation",
            "value": "Portrait"
        }
    ]
}
Using vgg19 in the optimization procedure.
L-BFGS | iteration: 000, total loss=2115127607296.0000, content_loss=      0.0000, style loss=2115117600000.0000, tv loss=10002612.0000
L-BFGS | iteration: 001, total loss=2115125903360.0000, content_loss=      0.0012, style loss=2115115920000.0000, tv loss=10002612.0000
L-